## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yulara,AU,2021-10-27 14:59:23,-25.2406,130.9889,76.84,22,67,0.00,broken clouds
1,1,Bredasdorp,ZA,2021-10-27 14:59:23,-34.5322,20.0403,76.28,80,100,15.68,light rain
2,2,Hamilton,US,2021-10-27 14:58:52,39.1834,-84.5333,50.99,81,66,1.01,broken clouds
3,3,Butaritari,KI,2021-10-27 14:59:24,3.0707,172.7902,82.29,72,100,13.58,light rain
4,4,Aklavik,CA,2021-10-27 14:59:24,68.2191,-135.0107,27.32,95,100,5.35,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yulara,AU,2021-10-27 14:59:23,-25.2406,130.9889,76.84,22,67,0.00,broken clouds
1,1,Bredasdorp,ZA,2021-10-27 14:59:23,-34.5322,20.0403,76.28,80,100,15.68,light rain
3,3,Butaritari,KI,2021-10-27 14:59:24,3.0707,172.7902,82.29,72,100,13.58,light rain
9,9,Champerico,GT,2021-10-27 14:59:26,14.3000,-91.9167,88.36,83,40,5.99,scattered clouds
12,12,Guarapari,BR,2021-10-27 14:54:37,-20.6667,-40.4975,84.09,70,75,13.35,broken clouds
14,14,Bambous Virieux,MU,2021-10-27 14:54:37,-20.3428,57.7575,75.45,69,20,10.36,few clouds
17,17,Koungou,YT,2021-10-27 14:59:28,-12.7336,45.2042,82.11,65,0,4.61,clear sky
18,18,Arraial Do Cabo,BR,2021-10-27 14:59:28,-22.9661,-42.0278,78.76,83,75,13.80,light rain
21,21,Hermanus,ZA,2021-10-27 14:59:29,-34.4187,19.2345,76.80,74,100,9.84,light rain
26,26,Ternate,ID,2021-10-27 14:59:31,0.8000,127.4000,81.32,78,14,3.49,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
city_data_df.count()

City_ID                693
City                   693
Country                689
Date                   693
Lat                    693
Lng                    693
Max Temp               693
Humidity               693
Cloudiness             693
Wind Speed             693
Current Description    693
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.dropna()
city_data_df.count()

City_ID                693
City                   693
Country                689
Date                   693
Lat                    693
Lng                    693
Max Temp               693
Humidity               693
Cloudiness             693
Wind Speed             693
Current Description    693
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Yulara,AU,76.84,broken clouds,-25.2406,130.9889,
1,Bredasdorp,ZA,76.28,light rain,-34.5322,20.0403,
2,Hamilton,US,50.99,broken clouds,39.1834,-84.5333,
3,Butaritari,KI,82.29,light rain,3.0707,172.7902,
4,Aklavik,CA,27.32,overcast clouds,68.2191,-135.0107,
5,Alofi,NU,73.29,scattered clouds,-19.0595,-169.9187,
6,Paamiut,GL,32.40,clear sky,61.9940,-49.6678,
7,Svetlaya,RU,44.44,overcast clouds,46.5389,138.3297,
8,Dudinka,RU,-3.06,overcast clouds,69.4058,86.1778,
9,Champerico,GT,88.36,scattered clouds,14.3000,-91.9167,


In [8]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()


City                   689
Country                689
Max Temp               689
Current Description    689
Lat                    689
Lng                    689
Hotel Name             689
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))